In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

from fastai import *
from fastai.vision import *
from fastai.vision.gan import *
from ArNet.generators import *
from ArNet.critics import *
from ArNet.dataset import *
from ArNet.loss import *
from ArNet.save import *
from ArNet.fid_loss import *
from ArNet.ssim import *
from ArNet.metrics import *

import torchvision
import geffnet # efficient/ mobile net

In [ ]:
path = Path('./dataset/')

path_fullRes = path/'DIV2K_train_HR'

path_lowRes_512 = path/'DIV2K_train_LR_512_QF20'

proj_id = 'unet_superRes_mobilenetV3_LPIPS'

gen_name = proj_id + '_gen'
crit_name = proj_id + '_crit'

name_gen = "MobilenetV3_Patch_GEN"
path_gen = path/name_gen

generate_training_img = False

# Super resolution eval

In [5]:
model = geffnet.mobilenetv3_rw
model = geffnet.mobilenetv3_small_minimal_100
loss_func = lpips_loss()

bs=1
sz=1024

path_lowRes_512 = path/'DIV2K_train_LR_512_QF20'

data_gen = get_DIV2k_data_QF(path_lowRes_512, path_fullRes, bs=bs, sz=sz)

learn_gen = gen_learner_wide(data=data_gen,
                             gen_loss=loss_func,
                             arch = model,
                             nf_factor=2)

Setting up Perceptual loss...
Loading model from: /data/students_home/fmameli/anaconda3/envs/fastai/lib/python3.6/site-packages/perceptual_similarity/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done


Learner(data=ImageDataBunch;

Train: LabelList (800 items)
x: ImageImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
y: ImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
Path: dataset/DIV2K_train_LR_512_QF20;

Valid: LabelList (100 items)
x: ImageImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
y: ImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
Path: dataset/DIV2K_train_LR_512_QF20;

Test: None, model=DynamicUnetWide(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Sequential(
        (0): Seque

In [6]:
learn_gen.metrics.append(SSIM_Metric_gen())
learn_gen.metrics.append(SSIM_Metric_input())
learn_gen.metrics.append(LPIPS_Metric_gen())
learn_gen.metrics.append(BRISQUE_Metric_gen())
learn_gen.metrics.append(BRISQUE_Metric_input())
learn_gen.metrics.append(BRISQUE_Metric_target())
learn_gen.metrics.append(NIQE_Metric_gen())
learn_gen.metrics.append(NIQE_Metric_input())
learn_gen.metrics.append(NIQE_Metric_target())

Setting up Perceptual loss...
Loading model from: /data/students_home/fmameli/anaconda3/envs/fastai/lib/python3.6/site-packages/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done


In [ ]:
weights = "/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/unet_wideNf2_mobileV3_DivFlickr1k_P64px_SuperRes_gen_3"
learn_gen.load(weights, with_opt=False)

In [5]:
val = learn_gen.validate()

In [6]:
for name, m in zip(learn_gen.metrics, val[1:]):
    if isinstance(m, torch.Tensor):
        print(name, m.item())
    else:
        print(name, m)

SSIM_Metric_gen 0.580901026725769
SSIM_Metric_input 0.5582197308540344
LPIPS_Metric_gen 0.36763280630111694
BRISQUE_Metric_gen 102.40903869282354
BRISQUE_Metric_input 103.40584471476105
BRISQUE_Metric_target 92.06918622940726
NIQE_Metric_gen 18.207478
NIQE_Metric_input 19.16361
NIQE_Metric_target 15.31811


In [ ]:
model = geffnet.mobilenetv3_small_minimal_100

learn_gen = gen_learner_wide(data=data_gen,
                             gen_loss=loss_func,
                             arch = model,
                             nf_factor=2)

weights = "/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/unet_wideNf2_mobileMin_DivFlickr1k_P64px_SuperRes_gen_3"
learn_gen.load(weights, with_opt=False)

In [7]:
val = learn_gen.validate()

In [8]:
for name, m in zip(learn_gen.metrics, val[1:]):
    if isinstance(m, torch.Tensor):
        print(name, m.item())
    else:
        print(name, m)

SSIM_Metric_gen 0.5830532312393188
SSIM_Metric_input 0.5582197308540344
LPIPS_Metric_gen 0.34275034070014954
BRISQUE_Metric_gen 102.2244990084559
BRISQUE_Metric_input 103.40584471476105
BRISQUE_Metric_target 92.06918622940726
NIQE_Metric_gen 18.386282
NIQE_Metric_input 19.16361
NIQE_Metric_target 15.31811


### MSE

In [4]:
model = geffnet.mobilenetv3_rw

learn_gen = gen_learner_wide(data=data_gen,
                             gen_loss=loss_func,
                             arch = model,
                             nf_factor=2)

weights = "/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/unet_superRes_mobilenetV3_MSE_gen_512px_2"
learn_gen.load(weights, with_opt=False)

learn_gen.metrics.append(SSIM_Metric_gen())
learn_gen.metrics.append(SSIM_Metric_input())
learn_gen.metrics.append(LPIPS_Metric_gen())
learn_gen.metrics.append(BRISQUE_Metric_gen())
learn_gen.metrics.append(BRISQUE_Metric_input())
learn_gen.metrics.append(BRISQUE_Metric_target())
learn_gen.metrics.append(NIQE_Metric_gen())
learn_gen.metrics.append(NIQE_Metric_input())
learn_gen.metrics.append(NIQE_Metric_target())

Setting up Perceptual loss...
Loading model from: /data/students_home/fmameli/anaconda3/envs/fastai/lib/python3.6/site-packages/perceptual_similarity/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done
Setting up Perceptual loss...
Loading model from: /data/students_home/fmameli/anaconda3/envs/fastai/lib/python3.6/site-packages/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done


In [5]:
val = learn_gen.validate()

In [6]:
for name, m in zip(learn_gen.metrics, val[1:]):
    if isinstance(m, torch.Tensor):
        print(name, m.item())
    else:
        print(name, m)

SSIM_Metric_gen 0.5680757164955139
SSIM_Metric_input 0.5582197308540344
LPIPS_Metric_gen 0.41735437512397766
BRISQUE_Metric_gen 103.1188594359636
BRISQUE_Metric_input 103.40584471476105
BRISQUE_Metric_target 92.06918622940726
NIQE_Metric_gen 18.479246
NIQE_Metric_input 19.16361
NIQE_Metric_target 15.31811


### SSIM

In [7]:
weights = "/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/unet_superRes_mobilenetV3_SSIM_gen_512px_0"
learn_gen.load(weights, with_opt=False)

Learner(data=ImageDataBunch;

Train: LabelList (800 items)
x: ImageImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
y: ImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
Path: dataset/DIV2K_train_LR_512_QF20;

Valid: LabelList (100 items)
x: ImageImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
y: ImageList
Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024),Image (3, 1024, 1024)
Path: dataset/DIV2K_train_LR_512_QF20;

Test: None, model=DynamicUnetWide(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): HardSwish()
      (3): Sequential(
        (0): Sequential(


In [8]:
val = learn_gen.validate()

In [9]:
for name, m in zip(learn_gen.metrics, val[1:]):
    if isinstance(m, torch.Tensor):
        print(name, m.item())
    else:
        print(name, m)

SSIM_Metric_gen 0.5877182483673096
SSIM_Metric_input 0.5582197308540344
LPIPS_Metric_gen 0.4079550802707672
BRISQUE_Metric_gen 100.28882333915725
BRISQUE_Metric_input 103.40584471476105
BRISQUE_Metric_target 92.06918622940726
NIQE_Metric_gen 18.223585
NIQE_Metric_input 19.16361
NIQE_Metric_target 15.31811
